In [32]:
import pandas as pd
import numpy as np
import jieba
import tensorflow.keras as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense
from sklearn.model_selection import train_test_split
from keras.layers import LSTM
from keras import layers
from keras import regularizers
from sklearn import metrics

In [33]:
df=pd.read_csv('train.csv',encoding = 'utf-8')
df=df.sample(frac=1).reset_index(drop=True)
df

,label\tcomment
0,0\t环境很惬意，菜很好吃！价格比较合理！
1,0\t超级差！服务差！配送差！口味差！点的东西带不完！直接来一句钱退你好了！
2,0\t感觉还是挺不错的，很好喝，然后，嗯，其实也没有便宜太多，和原价相比一块钱左右吧！哦，就...
3,0\t一如既往的实惠
4,0\t味道不错，挺好吃的，分量也足，关键是佐料打得好，看起都安逸
...,...
9995,0\t囗味细，味道好。性价比高。
9996,0\t味道不错，馅也多，不想有些地方根本吃不到馅，推荐。
9997,0\t服务态度很好，烧烤味道很好吃。而去好划算！
9998,0\t亲 味道真不好吃，


In [34]:
x_train=[]
y_train=[]
x=[]

In [35]:
for i in range(len(df)):
     x.append(df['label\tcomment'][i].split('\t')[1])
     x_train.append(' '.join(jieba.cut(x[i],cut_all=False)))
     y_train.append(df['label\tcomment'][i].split('\t')[0])

In [36]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [37]:
stop_words_file = "哈工大停用词表.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [38]:
fil_x=[]
for i in range(len(x_train)):
    q=x_train[i].split()
    strr=''
    for word in q:
        if word not in stopwords:
            strr=strr+word+' '
    fil_x.append(strr)

In [39]:
data_y=np.asarray(y_train)

In [40]:
num_words =12000   #设置的最大词数
max_len=50

tk = Tokenizer(num_words=num_words+1)  #因为要加未登录词，所以+1

tk.fit_on_texts(fil_x)
tk.word_index[tk.oov_token] = num_words + 1 
sequence=tk.texts_to_sequences(fil_x)
data_x=pad_sequences(sequence,maxlen=max_len)

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(data_x, data_y, random_state=1,train_size=0.8,test_size=0.2)

In [42]:
dff=pd.read_csv('test_new.csv',encoding = 'utf-8')
id=[]
x_test=[]
x1=[]
for i in range(len(dff)):
     x1.append(dff['comment'][i])
     id.append(dff['id'][i])
     x_test.append(' '.join(jieba.cut(x1[i],cut_all=False)))

In [43]:
fil_x_test=[]
for i in range(len(x_test)):
    q=x_test[i].split()
    strr=''
    for word in q:
        if word not in stopwords:
            strr=strr+word+' '
    fil_x_test.append(strr)

In [44]:
sequence1=tk.texts_to_sequences(fil_x_test)
data_x_test=pad_sequences(sequence1,maxlen=max_len)

In [45]:
model=Sequential()
model.add(Embedding(num_words,120,input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(50,dropout=0.1,recurrent_dropout=0.5)))#output_dim=60
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

history=model.fit(X_train,Y_train,epochs=20,batch_size=64,validation_data=(X_test,Y_test))

AttributeError: module 'tensorflow_core.keras' has no attribute 'keras'

In [ ]:
history_dict=history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
history_dict=history.history
loss_values=history_dict['loss']
val_loss_values=history_dict['val_loss']
epochs=range(1,len(loss_values)+1)
plt.plot(epochs,loss_values,'bo',label='Training Loss')
plt.plot(epochs,val_loss_values,'b',label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc=history_dict['acc']
val_acc=history_dict['val_acc']
plt.plot(epochs,acc,'bo',label="Training acc")
plt.plot(epochs,val_acc,label='Validation acc')
plt.title("Traing and Validation accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model=Sequential()
model.add(Embedding(num_words,120,input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(50,dropout=0.1,recurrent_dropout=0.5)))#output_dim=60
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history=model.fit(X_train,Y_train,epochs=8,batch_size=64,validation_data=(X_test,Y_test))

In [ ]:
result=model.evaluate(X_test,Y_test)
result

In [ ]:
features1=model.predict(data_x_test)
output1=[]
for i in range(len(features1)):
    if features1[i][0]>0.5 or features1[i][0]==0.5:
        output1.append(1)
    else :
        output1.append(0)
        

In [ ]:
import pandas as pd
df3=pd.read_csv("correct.csv",encoding='utf-8')
df3=df3['label']
df3array=df3.values
df3array

In [ ]:
TN =0
TP=0
FP=0
FN=0

for i in range(len(output1)):
    if (output1[i]==0)and(int(df3array[i])==0):
        TN=TN+1
    if (output1[i]==1)and(int(df3array[i])==1):
        TP=TP+1
    if (output1[i]==0)and(int(df3array[i])==1):
        FN=FN+1
    if (output1[i]==1)and(int(df3array[i])==0):
        FP=FP+1;
acc=(TP/(TP+FP))
callbackrate=TP/(TP+FN)
score=(2*acc*callbackrate)/(acc+callbackrate)
print(score)

In [ ]:
(output1)

In [ ]:
print(len(id))
print(len(output1))

In [ ]:
dataFrameToSave=pd.DataFrame({'id':id,'label':output1})
dataFrameToSave.shape
dataFrameToSave.to_csv("sample.csv",index=False,sep=",")

In [ ]:
dataFrameToSave